In [1]:
import tkinter as tk
from tkinter import messagebox

class VendingMachine:
    def __init__(self):
        self.state = 'IDLE'
        self.stock = {'Soda': {'quantity': 2, 'price': 1.00},
                      'Chips': {'quantity': 1, 'price': 0.75},
                      'Candy': {'quantity': 0, 'price': 0.50}}
        self.selected_product = None
        self.inserted_amount = 0.0

    def insert_coin(self, amount):
        if self.state == 'IDLE':
            if amount > 0:
                self.state = 'INSERTING_COIN' # New intermediate state
                self.inserted_amount += amount
                return f"{amount:.2f} inserted. Current amount: {self.inserted_amount:.2f}. Please select a product."
            return "Invalid coin amount."
        elif self.state == 'INSERTING_COIN' or self.state == 'SELECTING_PRODUCT':
            if amount > 0:
                self.inserted_amount += amount
                return f"{amount:.2f} inserted. Current amount: {self.inserted_amount:.2f}."
            return "Invalid coin amount."
        return "Cannot insert coin at this time."

    def select_product(self, product):
        if self.state not in ['INSERTING_COIN', 'SELECTING_PRODUCT']:
            return "Insert coins first."
        if product not in self.stock:
            return "Invalid product selected."
        if self.stock[product]['quantity'] == 0:
            self.state = 'OUT_OF_STOCK'
            return f"{product} is out of stock."
        if self.inserted_amount < self.stock[product]['price']:
            return f"Not enough money. {self.stock[product]['price']:.2f} required for {product}."
        self.selected_product = product
        self.state = 'DISPENSING'
        return f"{product} selected. Price: {self.stock[product]['price']:.2f}. Click 'Dispense' to get your product."

    def dispense(self):
        if self.state == 'DISPENSING':
            change = self.inserted_amount - self.stock[self.selected_product]['price']
            self.stock[self.selected_product]['quantity'] -= 1
            product_dispensed = self.selected_product
            self.selected_product = None
            self.inserted_amount = 0.0
            self.state = 'IDLE'
            if change > 0:
                return f"{product_dispensed} dispensed. Change: {change:.2f}. Thank you!"
            else:
                return f"{product_dispensed} dispensed. Thank you!"
        return "No product to dispense."

    def refill(self, product, amount):
        if product in self.stock:
            self.stock[product]['quantity'] += amount
            if self.state == 'OUT_OF_STOCK' and self.stock[product]['quantity'] > 0:
                self.state = 'IDLE'
            return f"{product} refilled by {amount}."
        return "Invalid product."

    def cancel(self):
        if self.state in ['INSERTING_COIN', 'SELECTING_PRODUCT', 'DISPENSING']:
            refund = self.inserted_amount
            self.inserted_amount = 0.0
            self.selected_product = None
            self.state = 'IDLE'
            return f"Transaction cancelled. Refund: {refund:.2f} returned."
        return "Nothing to cancel."

    def get_state(self):
        return self.state

    def get_stock(self):
        stock_info = {}
        for product, details in self.stock.items():
            stock_info[product] = details['quantity']
        return stock_info.copy()

    def get_product_price(self, product):
        if product in self.stock:
            return self.stock[product]['price']
        return None

    def get_inserted_amount(self):
        return self.inserted_amount

class VendingMachineGUI:
    def __init__(self, master):
        self.vm = VendingMachine()
        self.master = master
        master.title("Vending Machine")
        master.geometry("400x600") # Increased height

        self.label = tk.Label(master, text="Welcome!", font=("Helvetica", 14))
        self.label.pack(pady=10)

        self.output = tk.Text(master, height=6, width=40, wrap=tk.WORD, bg="#f0f0f0")
        self.output.pack(pady=5)

        self.inserted_amount_label = tk.Label(master, text="Inserted: 0.00", font=("Helvetica", 12))
        self.inserted_amount_label.pack(pady=5)

        tk.Label(master, text="Insert Coins:").pack()
        coin_frame = tk.Frame(master)
        coin_frame.pack()
        coins = [0.25, 0.50, 1.00]
        for coin in coins:
            btn = tk.Button(coin_frame, text=f"${coin:.2f}", width=8,
                            command=lambda c=coin: self.insert_coin(c), bg="gold")
            btn.pack(side=tk.LEFT, padx=5)

        tk.Label(master, text="Select Product:").pack(pady=5)
        for product, details in self.vm.stock.items():
            btn_text = f"{product} (${details['price']:.2f})"
            btn = tk.Button(master, text=btn_text, width=20,
                            command=lambda p=product: self.select_product(p))
            btn.pack(pady=2)

        self.dispense_button = tk.Button(master, text="Dispense", command=self.dispense, width=20, bg="lightgreen", state=tk.DISABLED)
        self.dispense_button.pack(pady=5)

        self.cancel_button = tk.Button(master, text="Cancel Transaction", command=self.cancel, width=20)
        self.cancel_button.pack(pady=5)

        self.stock_button = tk.Button(master, text="View Stock", command=self.show_stock, width=20)
        self.stock_button.pack(pady=5)

        refill_frame = tk.Frame(master)
        refill_frame.pack(pady=5)
        tk.Label(refill_frame, text="Refill Options:").pack()
        refill_buttons = [("Candy", 3), ("Soda", 3), ("Chips", 3)]
        for product, amount in refill_buttons:
            btn = tk.Button(refill_frame, text=f"Refill {product} (+{amount})",
                            command=lambda p=product, a=amount: self.refill(p, a), width=20)
            btn.pack(pady=2)

        self.state_label = tk.Label(master, text="Current State: IDLE", font=("Helvetica", 12))
        self.state_label.pack(pady=10)

    def update_output(self, message):
        self.output.delete(1.0, tk.END)
        self.output.insert(tk.END, message)
        self.state_label.config(text=f"Current State: {self.vm.get_state()}")
        self.inserted_amount_label.config(text=f"Inserted: {self.vm.get_inserted_amount():.2f}")
        if self.vm.get_state() == 'DISPENSING':
            self.dispense_button.config(state=tk.NORMAL)
        else:
            self.dispense_button.config(state=tk.DISABLED)

    def insert_coin(self, amount):
        result = self.vm.insert_coin(amount)
        self.update_output(result)

    def select_product(self, product):
        result = self.vm.select_product(product)
        self.update_output(result)

    def dispense(self):
        result = self.vm.dispense()
        self.update_output(result)

    def cancel(self):
        result = self.vm.cancel()
        self.update_output(result)

    def show_stock(self):
        stock = self.vm.get_stock()
        message = "Current Stock:\n" + "\n".join(f"{k}: {v}" for k, v in stock.items())
        messagebox.showinfo("Stock Info", message)

    def refill(self, product, amount):
        result = self.vm.refill(product, amount)
        self.update_output(result)

if __name__ == '__main__':
    import sys
    if len(sys.argv) > 1 and sys.argv[1] == 'test':
        import unittest

        class TestVendingMachine(unittest.TestCase):
            def setUp(self):
                self.vm = VendingMachine()

            def test_initial_state(self):
                self.assertEqual(self.vm.get_state(), 'IDLE')

            def test_insert_coin(self):
                result = self.vm.insert_coin(0.50)
                self.assertEqual(self.vm.get_state(), 'INSERTING_COIN')
                self.assertEqual(self.vm.get_inserted_amount(), 0.50)
                self.assertIn("0.50 inserted", result)

            def test_out_of_stock_product(self):
                self.vm.insert_coin(1.00)
                result = self.vm.select_product('Candy')  # Candy is out of stock
                self.assertEqual(self.vm.get_state(), 'OUT_OF_STOCK')
                self.assertIn("out of stock", result)

            def test_successful_dispense(self):
                self.vm.insert_coin(1.00)
                self.vm.select_product('Soda')
                result = self.vm.dispense()
                self.assertEqual(self.vm.get_state(), 'IDLE')
                self.assertEqual(self.vm.stock['Soda']['quantity'], 1)
                self.assertIn("Soda dispensed", result)
                self.assertEqual(self.vm.get_inserted_amount(), 0.0)

            def test_successful_dispense_with_change(self):
                self.vm.insert_coin(1.50)
                self.vm.select_product('Soda')
                result = self.vm.dispense()
                self.assertEqual(self.vm.get_state(), 'IDLE')
                self.assertEqual(self.vm.stock['Soda']['quantity'], 1)
                self.assertIn("Soda dispensed", result)
                self.assertIn("Change: 0.50", result)
                self.assertEqual(self.vm.get_inserted_amount(), 0.0)

            def test_not_enough_money(self):
                self.vm.insert_coin(0.50)
                result = self.vm.select_product('Soda')
                self.assertIn("Not enough money", result)
                self.assertEqual(self.vm.get_state(), 'SELECTING_PRODUCT')
                self.assertEqual(self.vm.get_inserted_amount(), 0.50)

            def test_cancel_transaction(self):
                self.vm.insert_coin(0.75)
                result = self.vm.cancel()
                self.assertEqual(self.vm.get_state(), 'IDLE')
                self.assertEqual(self.vm.get_inserted_amount(), 0.0)
                self.assertIn("Refund: 0.75", result)

            def test_refill_product(self):
                self.vm.select_product('Candy') # Out of stock
                self.vm.refill('Candy', 2)
                self.assertEqual(self.vm.stock['Candy']['quantity'], 2)
                self.assertEqual(self.vm.get_state(), 'IDLE')

        unittest.main(argv=['first-arg-is-ignored'], exit=False)
    else:
        root = tk.Tk()
        app = VendingMachineGUI(root)
        root.mainloop()